In [ ]:
# TODO:
#    1. early stopping not working optuna experiment

In [ ]:
RUN_EXP = True
NUM_TRIALS = 3

In [ ]:
import os
import shutil
import pickle
import random
import sys
import numpy as np
import pandas as pd
import duckdb
from typing import List

import requests
from io import StringIO

import ibis
import ibis.selectors as s
from ibis import _
ibis.options.interactive = True

from sklearn.preprocessing import RobustScaler

import torch

from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import (
    Scaler,
    MissingValuesFiller,
    Mapper,
    InvertibleMapper,
)
from darts.dataprocessing import Pipeline
from darts.metrics import mape, smape, mae, ope, rmse
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, IceCreamHeaterDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression, GumbelLikelihood, GaussianLikelihood

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    TFTModel,
    TiDEModel,
    DLinearModel,
    NLinearModel,
    TSMixerModel
)


from torchmetrics import (
    SymmetricMeanAbsolutePercentageError, 
    MeanAbsoluteError, 
    MeanSquaredError,
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import mlflow

import warnings
warnings.filterwarnings("ignore")

# logging
import logging

# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
    plot_pareto_front,
)

In [ ]:
os.chdir('../..')

In [ ]:
# custom modules
import src.data_engineering as de
from src import params
from src import plotting
from src import utils
from src.modeling import get_ci_err, build_fit_tsmixerx, log_pretty

## will be loaded from root when deployed
from src.darts_wrapper import DartsGlobalModel

In [ ]:
log.info(f'FORECAST_HORIZON: {params.FORECAST_HORIZON}')
log.info(f'INPUT_CHUNK_LENGTH: {params.INPUT_CHUNK_LENGTH}')

## Data prep

In [ ]:
# connect to database
con = ibis.duckdb.connect("data/spp.ddb", read_only=True)
con.list_tables()

In [ ]:
lmp = de.prep_lmp(con)
lmp

In [ ]:
mtrf = de.prep_mtrf(con)
mtrf

In [ ]:
mtlf = de.prep_mtlf(con)
mtlf

In [ ]:
all_df = de.prep_all_df(con)
all_df

In [ ]:
all_df_pd = de.all_df_to_pandas(de.prep_all_df(con))
all_df_pd

In [ ]:
all_df_pd.info()

## Prep model training data

In [ ]:
lmp_all, train_all, test_all = de.get_train_test_all(con)

In [ ]:
all_series = de.get_all_series(lmp_all)
all_series[0].plot()

In [ ]:
train_series = de.get_train_series(train_all)
train_series[0].plot()

In [ ]:
test_series = de.get_test_series(test_all)
test_series[0].plot()

In [ ]:
futr_cov = de.get_futr_cov(all_df_pd)
futr_cov[0].plot()

In [ ]:
past_cov = de.get_past_cov(all_df_pd)
past_cov[0].plot()

In [ ]:
con.disconnect()

## Set up hyperparameter tuning study

https://unit8co.github.io/darts/examples/17-hyperparameter-optimization.html?highlight=optuna

In [ ]:
# test build fit function
model = build_fit_tsmixerx(
    series=train_series,
    val_series=test_series,
    future_covariates=futr_cov,
    past_covariates=past_cov,
    n_epochs=1)

In [ ]:
model.MODEL_TYPE

In [ ]:
model.model_params

In [ ]:
err_metric = model.backtest(
        series=test_series,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        retrain=False,
        forecast_horizon=params.FORECAST_HORIZON,
        stride=25,
        metric=[mae],
        verbose=False,
    )

err_metric = np.mean(err_metric)
err_metric 

In [ ]:
preds = model.predict(
        series=train_series, 
        n=params.FORECAST_HORIZON,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        num_samples=200,
)

errs = rmse(test_series, preds, n_jobs=-1, verbose=True)
errs = np.mean(errs)
errs

In [ ]:
get_ci_err(test_series, preds)

In [ ]:
np.mean(get_ci_err(test_series, preds))

In [ ]:
### test custom metric
val_backtest = model.backtest(
    series=test_series,
    past_covariates=past_cov,
    future_covariates=futr_cov,
    retrain=False,
    forecast_horizon=params.FORECAST_HORIZON,
    stride=25,
    metric=[mae, get_ci_err],
    verbose=False,
    num_samples=200,
    # retrain=False,
)
val_backtest

In [ ]:
errs = np.mean([e[0] for e in val_backtest])
errs

In [ ]:
ci_err = np.mean([e[1] for e in val_backtest])
ci_err

In [ ]:
def objective_tsmixer(trial):
    callback = [PyTorchLightningPruningCallback(trial, monitor="val_loss")]

    # Hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 16, 16)
    ff_size = trial.suggest_int("ff_size", 16, 16)
    num_blocks = trial.suggest_int("num_blocks", 16, 16)
    lr = trial.suggest_float("lr", 5e-5, 2e-4, step=1e-5)
    n_epochs = trial.suggest_int("n_epochs", 4, 10)
    dropout = trial.suggest_float("dropout", 0.25, 0.5, step=0.01)
    

    # build and train the TCN model with these hyper-parameters:
    model = build_fit_tsmixerx(
        series=train_series,
        val_series=test_series,
        future_covariates=futr_cov,
        past_covariates=past_cov,
        hidden_size=hidden_size,
        ff_size=ff_size,
        num_blocks=num_blocks,
        lr=lr,
        n_epochs=n_epochs,
        dropout=dropout, 
        callbacks=callback,
    )

    # Evaluate how good it is on the validation set
    val_backtest = model.backtest(
        series=test_series,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        retrain=False,
        forecast_horizon=params.FORECAST_HORIZON,
        stride=25,
        metric=[mae, get_ci_err],
        verbose=False,
        num_samples=200,
        # retrain=False,
    )
    
    err_metric = np.mean([e[0] for e in val_backtest])
    ci_error = np.mean([e[1] for e in val_backtest])
    
    if err_metric!= np.nan:
        pass
    else:
        err_metric = float("inf")

    if ci_error!= np.nan:
        pass
    else:
        ci_error = float("inf")
    

    return err_metric , ci_error



In [ ]:
def print_callback(study, trial):
    best_smape = min(study.best_trials, key=lambda t: t.values[0])
    best_ci = min(study.best_trials, key=lambda t: t.values[1])
    best_total = min(study.best_trials, key=lambda t: sum(t.values))
    print('\n' + '*'*30, flush=True)
    log.info(f"\nTrial: {trial.number} Current values: {trial.values}")
    log.info(f"Current params: \n{log_pretty(trial.params)}")
    log.info(f"Best {target_names[0]}: {best_smape.values}, Best params: \n{log_pretty(best_smape.params)}")
    log.info(f"Best {target_names[1]}: {best_ci.values}, Best params: \n{log_pretty(best_ci.params)}")
    log.info(f"Best Total: {best_total.values}, Best params: \n{log_pretty(best_total.params)}")

In [ ]:
target_names = ['MAE', 'CI_ERROR']

## Start Experiment

In [ ]:
study = optuna.create_study(
    directions=["minimize", "minimize"],
    storage="sqlite:///spp_trials.db", 
    study_name="spp_weis_tsmixer",
    load_if_exists=True,
)
objective_func = objective_tsmixer

In [ ]:
study.optimize(objective_func, n_trials=NUM_TRIALS, callbacks=[print_callback])

In [ ]:
for i, target_name in enumerate(target_names):
    fig = plot_optimization_history(study, target=lambda t: t.values[i], target_name=target_name)
    fig.show()

In [ ]:
for i, target_name in enumerate(target_names):
    fig = plot_contour(study, params=["lr", "n_epochs"], target=lambda t: t.values[i], target_name=target_name)
    fig.show()

In [ ]:
plot_param_importances(study)

In [ ]:
plot_pareto_front(study, target_names=target_names)

In [ ]:
plot_pareto_front(study, target_names=target_names, include_dominated_trials=False)

In [ ]:
# TODO: think about how to wieght values to get best model
# best_model = min(study.best_trials, key=lambda t: sum(t.values))
best_model = min(study.best_trials, key=lambda t: t.values[0] + 0.5*t.values[1])
log.info(f"Best number: {best_model.number}")
log.info(f"Best values: {best_model.values}")
log.info(f"Best params: \n{log_pretty(best_model.params)}")

## MLFlow setup

In [ ]:
# mlflow.set_tracking_uri("sqlite:///mlruns.db")
log.info(f'mlflow.get_tracking_uri(): {mlflow.get_tracking_uri()}')
exp_name = 'spp_weis'

if mlflow.get_experiment_by_name(exp_name) is None:
    exp = mlflow.create_experiment(exp_name)
    
exp = mlflow.get_experiment_by_name(exp_name)
exp

## Get model signature

In [ ]:
node_series = train_series[0]
future_cov_series = futr_cov[0]
past_cov_series = past_cov[0]

data = {
    'series': [node_series.to_json()],
    'past_covariates': [past_cov_series.to_json()],
    'future_covariates': [future_cov_series.to_json()],
    'n': params.FORECAST_HORIZON,
    'num_samples': 200
}

df = pd.DataFrame(data)

ouput_example = 'the endpoint return json as a string'

from mlflow.models import infer_signature
darts_signature = infer_signature(df, ouput_example)
darts_signature

## Refit and log model with best params

In [ ]:
with mlflow.start_run(experiment_id=exp.experiment_id) as run:
    
    # fit model with best params from study
    model = build_fit_tsmixerx(
        series=train_series,
        val_series=test_series,
        future_covariates=futr_cov,
        past_covariates=past_cov,
        **best_model.params
    )
    
    log.info(f'run.info: \n{run.info}')
    artifact_path = "model_artifacts"
    metrics = {}
    model_params = model.model_params
    
    # back test on validation data
    acc = model.backtest(
        series=test_series,
        # series=all_series,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        retrain=False,
        forecast_horizon=model_params['output_chunk_length'],
        stride=25,
        metric=[mae, rmse, get_ci_err],
        verbose=False,
        num_samples=200,
    )

    # log.info(f'BACKTEST: acc: {acc}')
    log.info(f'BACKTEST: np.mean(acc, axis=0): {np.mean(acc, axis=0)}')
    acc_df = pd.DataFrame(
        np.mean(acc, axis=0).reshape(1,-1),
        columns=['mae', 'rmse', 'ci_error']
    )

    # add metrics
    metrics['test_mae'] = acc_df.mae[0]
    metrics['test_rmse'] = acc_df.rmse[0]
    metrics['test_ci_error'] = acc_df.ci_error[0]

    # final training
    final_train_series = test_series
    log.info('final training')
    model.fit(
            series=test_series,
            past_covariates=past_cov,
            future_covariates=futr_cov,
            verbose=True,
            )
    
    # final model back test on validation data
    acc = model.backtest(
            series=test_series,
            past_covariates=past_cov,
            future_covariates=futr_cov,
            retrain=False,
            forecast_horizon=model_params['output_chunk_length'],
            stride=25,
            metric=[mae, rmse, get_ci_err],
            verbose=False,
            num_samples=200,
        )

    log.info(f'FINAL ACC: np.mean(acc, axis=0): {np.mean(acc, axis=0)}')
    acc_df = pd.DataFrame(
        np.mean(acc, axis=0).reshape(1,-1),
        columns=['mae', 'rmse', 'ci_error']
    )

    # add and log metrics
    metrics['final_mae'] = acc_df.mae[0]
    metrics['final_rmse'] = acc_df.rmse[0]
    metrics['final_ci_error'] = acc_df.ci_error[0]
    mlflow.log_metrics(metrics)

    # set up path to save model
    model_path = '/'.join([artifact_path, model.MODEL_TYPE])

    shutil.rmtree(artifact_path, ignore_errors=True)
    os.makedirs(artifact_path)

    # log params
    mlflow.log_params(model_params)

    # save model files (model, model.ckpt) 
    # and load them to artifacts when logging the model
    model.save(model_path)

    # save MODEL_TYPE to artifacts
    # this will be used to load the model from the artifacts
    model_type_path = '/'.join([artifact_path, 'MODEL_TYPE.pkl'])
    with open(model_type_path, 'wb') as handle:
        pickle.dump(model.MODEL_TYPE, handle)

    model_type_timestamp = '/'.join([artifact_path, 'TRAIN_TIMESTAMP.pkl'])
    with open(model_type_timestamp, 'wb') as handle:
        pickle.dump(model.TRAIN_TIMESTAMP, handle)
    
    # map model artififacts in dictionary
    artifacts = {
        'model': model_path,
        'model.ckpt': model_path+'.ckpt',
        'MODEL_TYPE': model_type_path,
    }
    
    # log model
    # https://www.mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#pip-requirements-example
    mlflow.pyfunc.log_model(
        artifact_path='GlobalForecasting',
        code_path=['notebooks/model_training/darts_wrapper.py'],
        signature=darts_signature,
        artifacts=artifacts,
        python_model=DartsGlobalModel(), 
        pip_requirements=["-r notebooks/model_training/requirements.txt"],
    )


## Get latest run and test predicting

In [ ]:
runs = mlflow.search_runs(
    experiment_ids = exp.experiment_id,
    # order_by=['metrics.test_mae']
    order_by=['end_time']
    )

runs.sort_values('end_time', ascending=False, inplace=True)
runs.head()

In [ ]:
best_run_id = runs.run_id.iloc[0]
best_run_id

In [ ]:
runs['artifact_uri'].iloc[0]

In [ ]:
model_path = runs['artifact_uri'].iloc[0] + '/GlobalForecasting'

In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_path)

## Plot test predictions

In [ ]:
plot_ind = 3
plot_series = all_series[plot_ind]

In [ ]:
plot_series.static_covariates.unique_id.LMP

In [ ]:
plot_series.plot()

In [ ]:
plot_end_times = pd.date_range(
    end=test_series[plot_ind].end_time(),
    periods=5,
    freq='d',
)

for plot_end_time in plot_end_times:
    plot_end_time = min(
        plot_series.end_time() - pd.Timedelta(f'{params.INPUT_CHUNK_LENGTH+1}h'), 
        pd.Timestamp(plot_end_time)
    )
    log.info(f'plot_end_time: {plot_end_time}')
    
    plot_node_name = plot_series.static_covariates.unique_id.LMP
    
    # if test_end_time < test_series.end_time():
    node_series = plot_series.drop_after(plot_end_time)
        
    log.info(f'plot_end_time: {plot_end_time}')
    log.info(f'node_series.end_time(): {node_series.end_time()}')
    future_cov_series = futr_cov[0]
    past_cov_series = past_cov[0]
    
    data = {
        'series': [node_series.to_json()],
        'past_covariates': [past_cov_series.to_json()],
        'future_covariates': [future_cov_series.to_json()],
        'n': params.FORECAST_HORIZON,
        'num_samples': 200
    }
    df = pd.DataFrame(data)
    
    plot_cov_df = future_cov_series.pd_dataframe()
    plot_cov_df = (
        plot_cov_df
        .reset_index()
        .rename(columns={'timestamp_mst':'time', 're_ratio': 'Ratio'})
    )
    
    # Predict on a Pandas DataFrame.
    df['num_samples'] = 500
    pred = loaded_model.predict(df)
    preds = TimeSeries.from_json(pred)
    
    q_df = plotting.get_quantile_df(preds)
    
    plot_df = plotting.get_mean_df(preds).merge(
        plotting.get_quantile_df(preds),
        left_index=True,
        right_index=True,
    )
    
    lmp_df = lmp.to_pandas().rename(
        columns={
            'LMP': 'LMP_HOURLY',
            'unique_id':'node', 
            'timestamp_mst':'time'
        })
    
    plot_df = plotting.get_plot_df(
            TimeSeries.from_json(pred),
            plot_cov_df,
            lmp_df,
            plot_node_name,
        )
    plot_df.rename(columns={'mean':'mean_fcast'}, inplace=True)
    plot_df
    
    plotting.plotly_forecast(plot_df, plot_node_name, show_fig=True)

In [ ]:
df